# Import Imp Libraries

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


# Load & Preprocess Data

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Update this path
dataset_path = "Your Path of Dataset"    


datagen = ImageDataGenerator(validation_split=0.2, rescale=1.0/255)

train_data = datagen.flow_from_directory(dataset_path, 
                                         target_size=(32, 32), 
                                         color_mode="grayscale",
                                         batch_size=64,
                                         class_mode="categorical",
                                         subset="training")

val_data = datagen.flow_from_directory(dataset_path, 
                                       target_size=(32, 32), 
                                       color_mode="grayscale",
                                       batch_size=64,
                                       class_mode="categorical",
                                       subset="validation")


Found 62560 images belonging to 46 classes.
Found 15640 images belonging to 46 classes.


#  Use CNN Model

In [5]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,1)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(46, activation='softmax')  # 46 Hindi characters
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train Model and Save model in .h5 format

In [5]:
model.fit(train_data, validation_data=val_data, epochs=2)
model.save("hindi_character_model.h5")


C:\Users\Am\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
978/978 ━━━━━━━━━━━━━━━━━━━━ 893s 907ms/step - accuracy: 0.6517 - loss: 1.2992 - val_accuracy: 0.8047 - val_loss: 0.6944
Epoch 2/2
978/978 ━━━━━━━━━━━━━━━━━━━━ 81s 82ms/step - accuracy: 0.9490 - loss: 0.1690 - val_accuracy: 0.8429 - val_loss: 0.6017


#  Real-time Character Prediction using OpenCV

In [6]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model("hindi_character_model.h5")

# Load label mappings
labels = {v: k for k, v in train_data.class_indices.items()}  

# Create a blank black canvas
canvas = np.zeros((400, 400), dtype=np.uint8)  # Black background
drawing = False 

# Mouse callback function
def draw(event, x, y, flags, param):
    global drawing, canvas
    if event == cv2.EVENT_LBUTTONDOWN:  # Start drawing
        drawing = True
    elif event == cv2.EVENT_MOUSEMOVE and drawing:  
        cv2.circle(canvas, (x, y), 10, 255, -1)  
    elif event == cv2.EVENT_LBUTTONUP:  # Stop drawing
        drawing = False

# Create window and set mouse callback
cv2.namedWindow("Draw Hindi Character")
cv2.setMouseCallback("Draw Hindi Character", draw)

while True:
    # Show drawing canvas
    cv2.imshow("Draw Hindi Character", canvas)

    key = cv2.waitKey(1)

    if key == ord('p'):  # Press 'p' to predict
        roi = canvas.copy()
        roi = cv2.resize(roi, (32, 32))  
        roi = roi / 255.0  # Normalize
        roi = np.reshape(roi, (1, 32, 32, 1))  

        # Predict character
        prediction = model.predict(roi)
        predicted_label = labels[np.argmax(prediction)]

        # Show result on the canvas
        print(f'Predicted Character: {predicted_label}')
        cv2.putText(canvas, f'Pred: {predicted_label}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, 255, 2)  

    elif key == ord('c'):  # Press 'c' to clear the canvas
        canvas = np.zeros((400, 400), dtype=np.uint8)

    elif key == ord('q'):  # Press 'q' to quit
        break

cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Predicted Character: ख


# NOTE :
       1. Read all code carefully then execute code step by step.
       2. After executing 1 code then read the comments carefully.
       3. The most important part is to set the correct path; otherwise code will not execute.
       4. Use a high-configuration Laptop or pc for better, faster execution.
       5. Understand the code, do not copy/paste it.
       